In [115]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

In [116]:
aoq1 = pd.read_csv(r'D:\Seksatta\MKG\summary lizard R&D\AOQ-VIS 010120-311223\aoq1_2.csv')
aoq2 = pd.read_csv(r'D:\Seksatta\MKG\summary lizard R&D\AOQ-VIS 010120-311223\aoq2.csv')
aoq3 = pd.read_csv(r'D:\Seksatta\MKG\summary lizard R&D\AOQ-VIS 010120-311223\aoq3.csv')

In [117]:
df = pd.concat([aoq1, aoq2, aoq3], keys=[1,2,3])

In [118]:
df.reset_index(inplace=True)

In [119]:
df.drop(columns = ['level_0', 'level_1'], inplace =True)


In [120]:
df.columns

Index(['SITE', 'FAC_MODEL', 'FAC_SERIAL', 'SYMPTOM_DESC',
       'FAILURE_PART_DESC_FINAL', 'FY_OF_CLAIM'],
      dtype='object')

In [121]:
df.drop_duplicates(subset = ['FAC_MODEL', 'FAC_SERIAL'], inplace=True)

In [122]:

df['SYMPTOM_DESC']=df['SYMPTOM_DESC'].str.replace(r'\b[dD]oest', 'Does', regex= True)
df['SYMPTOM_DESC']=df['SYMPTOM_DESC'].str.replace(r' /', '/', regex= True)
df['SYMPTOM_DESC']=df['SYMPTOM_DESC'].str.replace(r' /', '/', regex= True).str.rstrip('.')
df['SYMPTOM_DESC']=df['SYMPTOM_DESC'].str.replace(r'A/C', '', regex= True)
df['SYMPTOM_DESC']=df['SYMPTOM_DESC'].str.strip()
df['FAC_MODEL']=df['FAC_MODEL'].str.strip()
df['SYMPTOM_DESC']=df['SYMPTOM_DESC'].str.upper()
df['SYMPTOM_DESC']=df['SYMPTOM_DESC'].str.replace(r'10 :  DOES NOT OPERATE', '10 : DOES NOT OPERATE', regex= False)

In [123]:
df = df[df['FAILURE_PART_DESC_FINAL'].str.contains(r'\bPCB\b', case=False, regex=True)]

# Mapping with model and model group

In [124]:
model = pd.read_csv(r'D:\Seksatta\EQG\dit_info\model2.csv')

In [125]:
df = pd.merge(df, model[['base_model','model_name','group','i_o']], left_on = 'FAC_MODEL', right_on = 'model_name', how='left')
df.drop(columns=['model_name'], inplace=True, axis= 1)

In [126]:
df = df.dropna(subset=['group'])

In [127]:
df1 = df.groupby(['FY_OF_CLAIM', 'group', 'i_o', 'SYMPTOM_DESC', 'FAILURE_PART_DESC_FINAL'])['SITE'].size().reset_index()


In [128]:
df1.rename(columns = {'SITE':'QTY'}, inplace=True)

## Checking the result 

In [129]:
df1.groupby(['FY_OF_CLAIM','group','i_o']).size()

FY_OF_CLAIM  group  i_o    
2019         PA     indoor      10
                    outdoor      6
             RA     indoor     106
                    outdoor     86
             SKY    indoor      46
                    outdoor     86
             VRV    indoor      26
                    outdoor     38
2020         PA     indoor      15
                    outdoor     19
             RA     indoor     169
                    outdoor    134
             SKY    indoor      73
                    outdoor    123
             VRV    indoor      55
                    outdoor     76
2021         PA     indoor       6
                    outdoor      6
             RA     indoor     121
                    outdoor    105
             SKY    indoor      47
                    outdoor     95
             VRV    indoor      32
                    outdoor     54
2022         PA     indoor       4
                    outdoor      7
             RA     indoor      71
                    outdoor

## Export the result

In [130]:
df1.to_excel(r'pcb_service.xlsx')